In [1]:
import openvino as ov
import torch
from datasets import load_dataset
from transformers import AutoImageProcessor, ResNetForImageClassification
import time

In [2]:
core = ov.Core()
core.available_devices

['CPU', 'NPU']

In [3]:
# Download resnet-50 model
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

In [4]:
# Convert to onnx format
torch.onnx.export(
    model,
    torch.randn(1, 3, 224, 224),
    "./model/resnet50.onnx",
    input_names=["input"],
    output_names=["output"],
)

/home/yata/Desktop/openvino-tutorial/.venv/lib/python3.11/site-packages/transformers/models/resnet/modeling_resnet.py:91: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if num_channels != self.num_channels:


In [5]:
# Load onnx model
ov_model = ov.convert_model("./model/resnet50.onnx")

# Convert to OpenVINO format
cpu_model = core.compile_model(ov_model, device_name="CPU")
# npu_model = core.compile_model(ov_model, device_name="NPU")

In [6]:
def create_dataset() -> list:
    dataset = load_dataset("Rapidata/Animals-10")
    processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")

    teset_dataset = []
    for i in range(len(dataset["train"])):
        image = dataset["train"][i]
        image_arrays = processor(image["image"], return_tensors="np")["pixel_values"]
        teset_dataset.append({"data": image_arrays, "label": image["label"]})

    return teset_dataset

In [7]:
teset_dataset = create_dataset()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [8]:
start = time.perf_counter()
for test_data in teset_dataset:
    cpu_model([test_data["data"]])
end = time.perf_counter()

print(f"CPU inference time: {end - start:.4f} seconds")

CPU inference time: 225.2637 seconds
